In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, CommaSeparatedListOutputParser, JsonOutputParser
from langchain_core.runnables import RunnablePassthrough
from pydantic import BaseModel, Field

In [4]:
# Initialize LLM
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0.7) # A bit higher temp for more varied responses

Question 1: Basic LLM Chain

In [5]:
prompt = PromptTemplate.from_template("What is the capital of {country}")
parser = StrOutputParser()
chain = prompt | llm | parser
result = chain.invoke({"country": "australia"})
print(result)


The capital of Australia is Canberra.


Question 2: Chain with Multiple Variables

In [6]:
prompt = PromptTemplate.from_template(
    "Translate the {text} from {source_lang} to {target_lang}"
    )
parser = StrOutputParser()
chain = prompt|llm|parser
result = chain.invoke({
    "text":"Where is the nearest cafe?",
    "source_lang":"English",
    "target_lang":"japanese"
    })
print(result)

The translation of "Where is the nearest cafe?" into Japanese is:

一番近いカフェはどこですか？  
(Ichiban chikai kafe wa doko desu ka?)


Question 3: Introducing CommaSeparatedListOutputParser

In [7]:
prompt = PromptTemplate.from_template(
    "List 3 common {fruits}, seperated by comma"
)
parser = CommaSeparatedListOutputParser()
chain = prompt|llm|parser
result = chain.invoke({"fruits":"tropical"})
print(result)

['Mango', 'banana', 'pineapple']


Question 4: Sequential Chain (Output of one is input to another)

In [8]:
concept_prompt = PromptTemplate.from_template(
    "Generate a very short, simple concept on mobile app on {category}."
)

expand_prompt = PromptTemplate.from_template(
    "Give this app concept: {concept}, provide 3 key features for it"
)

parser = StrOutputParser()
chain = (
    {"concept" : concept_prompt | llm | parser}
    | expand_prompt
    | llm
    | parser
)

response = chain.invoke({"category": "pet care"})
print(f"Chain Response (App concept + features):\n{response}")


Chain Response (App concept + features):
Sure! Here are 3 key features for **PetPal**:

1. **Customizable Care Schedules:**  
   Users can create and personalize daily routines for each pet, including feeding times, walks, medication doses, grooming, and other care tasks.

2. **Smart Reminders & Notifications:**  
   The app sends timely alerts and push notifications to remind users of upcoming tasks, ensuring no care activity is missed.

3. **Activity Logging & History:**  
   Users can easily log completed tasks and view a history of their pet’s care activities, helping them track consistency and share updates with vets or pet sitters.


Question 5: ChatPromptTemplate in a Chain

In [9]:
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an expert on {topic}. Provide a concise answer."),
    ("human", "{question}"),
])

parser = StrOutputParser()
chain = chat_prompt|llm|parser
result = chain.invoke({
    "topic" : "data science",
    "question" : "what is random forest"
})
print(result)

Random Forest is an ensemble machine learning algorithm used for classification and regression tasks. It builds multiple decision trees during training and outputs the mode of the classes (classification) or mean prediction (regression) of the individual trees. This approach improves accuracy and reduces overfitting compared to a single decision tree.


Question 6: JsonOutputParser with Pydantic Model

In [10]:
class BookInfo(BaseModel):
    title:str = Field(description="The title of the book")
    author:str = Field(description="The author of the book")
    genre:str = Field(description="The genre of the book")
    publication_year:int = Field(description="The year the book was published")

json_parser = JsonOutputParser(pydantic_object = BookInfo)
json_schema = json_parser.get_format_instructions()

json_prompt = PromptTemplate.from_template(
    "Extract information about the book '{book_name}.\n"
    "Format your output as JSON, according to the following schema: \n{format_instructions}"
)
chain = json_prompt|llm|json_parser

response = chain.invoke({
    "book_name":"The Hitchhiker's Guide to the Galaxy",
    "format_instructions" : json_schema
})
print(response)

{'title': "The Hitchhiker's Guide to the Galaxy", 'author': 'Douglas Adams', 'genre': 'Science Fiction', 'publication_year': 1979}


Question 7: More Complex Sequential Chain with Context Preservation

In [20]:
parser = StrOutputParser()
# Step 1: Summarize the sentence
summary_prompt = PromptTemplate.from_template("Summarize this: {sentence}")
summary_chain = summary_prompt | llm | parser

# Step 2: Combine original and summary
combo_prompt = PromptTemplate.from_template(
    "Original: {sentence}\nSummary: {summary}\nWrite a joke about this topic."
)
combo_chain = combo_prompt | llm | parser

# Chain with pass-through for context
chain = (
    {"sentence": RunnablePassthrough()}
    | {"summary": summary_chain, "sentence": RunnablePassthrough()}
    | combo_chain
)

result = chain.invoke({"sentence": "Otters hold hands while sleeping so they don't drift apart."})
print(result)

Why did the otters hold hands while sleeping?  
Because they didn’t want their dreams to drift apart!


Question 8: Simple Retrieval Chain

In [21]:
parser = StrOutputParser()

def dummy_retriever(query:str):
    if "python" in query.lower():
        return "Python is a high-level, interpreted programming language. It is known for its simplicity and readability. Common uses include web development, data analysis, AI, and scripting."
    elif "javascript" in query.lower():
        return "JavaScript is a programming language that is one of the core technologies of the World Wide Web, alongside HTML and CSS. It enables interactive web pages and is an essential part of web applications."
    else:
        return "No specific information found for this query."

retrieval_prompt = PromptTemplate.from_template(
    "Given the following information:\n{context}\n\nAnswer the questions: {query}"
)

chain = (
    RunnablePassthrough.assign(context=lambda x: dummy_retriever(x["query"])) # Run retriever to get context
    | retrieval_prompt
    | llm
    | parser
)

response_python = chain.invoke({"query": "What are the common uses of Python?"})
print(f"Retrieval Chain (Python): {response_python}\n")

response_js = chain.invoke({"query": "What is JavaScript primarily used for on the web?"})
print(f"Retrieval Chain (JavaScript): {response_js}")

Retrieval Chain (Python): The common uses of Python are web development, data analysis, AI, and scripting.

Retrieval Chain (JavaScript): JavaScript is primarily used on the web to enable interactive web pages and create dynamic, responsive user experiences within web applications.
